# ISTE-DL-SGP-21 WEEK 4 ASSIGNMENT


## Importing Libraries 

In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam

## Preprocessing the Dataset

In [4]:
DATADIR = '../input/covidistesgp/CovidDataset'

TRAIN_DIR = os.path.join(DATADIR, 'train')
TEST_DIR = os.path.join(DATADIR, 'validation')

### Image Data Generator

In [5]:
aug = ImageDataGenerator(
    rotation_range=7,
    width_shift_range=0.5,
    height_shift_range=0.5,
    brightness_range=(0.9, 1.1),
    shear_range=2,
    zoom_range=0.1,
    fill_mode='nearest',
    channel_shift_range=1,
    horizontal_flip=True,
    vertical_flip=True,
    data_format='channels_last',
    rescale=1/255,
    validation_split=0.3,
)

load = ImageDataGenerator(
    rescale=1/255
)

In [6]:
IMG_SIZE=299

train_datagen = aug.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    shuffle=True,
    class_mode='binary',
    color_mode='rgb',
    subset='training',
)
validation_datagen = aug.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=8,
    shuffle=True,
    class_mode='binary',
    color_mode='rgb',
    subset='validation',
)

test_datagen = load.flow_from_directory(
    TEST_DIR,
    class_mode='binary',
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='rgb',
)

Found 1400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


## Transfer Learning

### Using InceptionV3

In [7]:
base = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

87916544/87910968 [==============================] - 1s 0us/step


In [8]:
model = Sequential()

model.add(base)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

#### Freezing InceptionV3 Params


In [9]:
base.trainable = False

In [10]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 inception_v3 False
1 global_average_pooling2d True
2 dropout True
3 dense True
4 dropout_1 True
5 dense_1 True


In [11]:
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(lr=0.001), 
    metrics=tf.keras.metrics.AUC()
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                131136    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 21,933,985
Trainable params: 131,201
Non-trainable params: 21,802,784
______________________________________

In [12]:
model.fit(train_datagen, epochs=5, validation_data=validation_datagen)

Epoch 1/5
44/44 [==============================] - 69s 1s/step - loss: 0.5437 - auc: 0.7400 - val_loss: 0.1637 - val_auc: 0.9806
Epoch 2/5
44/44 [==============================] - 48s 1s/step - loss: 0.2618 - auc: 0.9424 - val_loss: 0.1674 - val_auc: 0.9812
Epoch 3/5
44/44 [==============================] - 47s 1s/step - loss: 0.2086 - auc: 0.9640 - val_loss: 0.1364 - val_auc: 0.9868
Epoch 4/5
44/44 [==============================] - 48s 1s/step - loss: 0.2090 - auc: 0.9671 - val_loss: 0.1813 - val_auc: 0.9859
Epoch 5/5
44/44 [==============================] - 47s 1s/step - loss: 0.2124 - auc: 0.9594 - val_loss: 0.1346 - val_auc: 0.9929


#### Unfreezing Base Layers

In [13]:
for layer in model.layers[-20:]:
    if not isinstance(layer, BatchNormalization):
        layer.trainable = True

In [14]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 inception_v3 True
1 global_average_pooling2d True
2 dropout True
3 dense True
4 dropout_1 True
5 dense_1 True


In [15]:
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(lr=0.0001), 
    metrics=tf.keras.metrics.AUC()
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                131136    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 21,933,985
Trainable params: 21,899,553
Non-trainable params: 34,432
_______________________________________

In [16]:
model.fit(train_datagen, epochs=5, validation_data=validation_datagen)

Epoch 1/5
44/44 [==============================] - 58s 1s/step - loss: 0.3288 - auc_1: 0.9304 - val_loss: 0.8932 - val_auc_1: 0.8267
Epoch 2/5
44/44 [==============================] - 50s 1s/step - loss: 0.1371 - auc_1: 0.9858 - val_loss: 0.1306 - val_auc_1: 0.9877
Epoch 3/5
44/44 [==============================] - 50s 1s/step - loss: 0.0918 - auc_1: 0.9911 - val_loss: 0.0755 - val_auc_1: 0.9962
Epoch 4/5
44/44 [==============================] - 50s 1s/step - loss: 0.1249 - auc_1: 0.9873 - val_loss: 0.1621 - val_auc_1: 0.9950
Epoch 5/5
44/44 [==============================] - 49s 1s/step - loss: 0.0909 - auc_1: 0.9929 - val_loss: 0.0956 - val_auc_1: 0.9898


## Score

In [17]:
score = model.evaluate(train_datagen)
print('Train Loss: ', score[0])
print('Train Accuracy: ', score[1])

44/44 [==============================] - 33s 741ms/step - loss: 0.1091 - auc_1: 0.9949
Train Loss:  0.1090945303440094
Train Accuracy:  0.9949184060096741


In [18]:
score = model.evaluate(test_datagen)
print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])

7/7 [==============================] - 3s 376ms/step - loss: 0.3630 - auc_1: 0.9761
Test Loss:  0.3630027770996094
Test Accuracy:  0.9761000275611877
